# Object Oriented Programming

## Custom classes


We've defined a Player class and set up the default __init__ method to accept a data row as an argument. We made a deliberate choice to split up the logic of players and teams so our code is easy to read and maintain. We also made the convenient choice to initialize our Player instances using a data row. That's because all of the information is present in a row, and it will make it easier to create Player objects from the data set later on.

In [2]:
import pandas as pd
nba = pd.read_csv("nba_players_2013.csv")

header=['player', 'pos', 'age', 'team']

print(nba.head())
nba = nba.values.tolist()
print(type(nba))

print(header)
print(nba[:5])
print(type(nba))

class Player():
    # The special __init__ function runs whenever a class is instantiated
    # The init function can take arguments, but self is always the first one
    # Self is just a reference to the instance of the class
    # It is automatically passed in when you instantiate an instance of the class
    def __init__(self, data_row):
        self.player_name = data_row[0]
        self.position = data_row[1]
        self.age = data_row[2]
        self.team = data_row[3]

# Initialize a player using the first row of our data set
first_player = Player(nba[0])

# Implement the Team class iterating over all players and adding them as Player classes:
class Team():
    def __init__(self, team_name):
        self.team_name = team_name
        # Team roster initially empty
        self.roster = []
        # Find the players for the roster in the data set
        for row in nba:
            if row[3] == "San Antonio Spurs":
                self.roster.append(Player(row))

                
spurs = Team("San Antonio Spurs")
print(spurs)


          player pos  age                   team   g  gs    mp   fg   fga  \
0     Quincy Acy  SF   23                    TOT  63   0   847   66   141   
1   Steven Adams   C   20  Oklahoma City Thunder  81  20  1197   93   185   
2    Jeff Adrien  PF   27                    TOT  53  12   961  143   275   
3  Arron Afflalo  SG   28          Orlando Magic  73  73  2552  464  1011   
4  Alexis Ajinca   C   25   New Orleans Pelicans  56  30   951  136   249   

     fg.     ...      drb  trb  ast  stl  blk  tov   pf   pts     season  \
0  0.468     ...      144  216   28   23   26   30  122   171  2013-2014   
1  0.503     ...      190  332   43   40   57   71  203   265  2013-2014   
2  0.520     ...      204  306   38   24   36   39  108   362  2013-2014   
3  0.459     ...      230  262  248   35    3  146  136  1330  2013-2014   
4  0.546     ...      183  277   40   23   46   63  187   328  2013-2014   

   season_end  
0        2013  
1        2013  
2        2013  
3        2013  


The Player and Team classes we've defined serve as blueprints that we can use to create instances of these classes. **Classes** and the **instances** of those classes, which are collectively known as **objects**, are fundamental to object-oriented programming.

## Instance methods

If we want to compute the **average age of the players on a team**, we would write a method for the Team class that does this. However, because this number can be different for each team, we want to make sure the method acts individually on specific instances of the Team class. We call these methods **instance methods**.
 
For method declarations, the **first argument to the method is always** `self`, even though we don't explicitly pass in self when we call the method. self is a reference to the current object we're working with. It's useful when we want to access properties of that object within the method we're defining.

In [3]:
class Team():
    def __init__(self, team_name):
        self.team_name = team_name
        # Team roster initially empty
        self.roster = []
        # Find the players for the roster in the data set
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count

    # Implement the average_age() instance method
    def average_age(self):
        age_sum = 0
        for player in self.roster:
            age_sum += player.age
        return age_sum / self.num_players()
            
            
spurs = Team("San Antonio Spurs")
spurs_num_players = spurs.num_players()

spurs_avg_age = spurs.average_age()
print(spurs_avg_age)

28.428571428571427


## Class methods

In traditional object-oriented programming, everything (yes, everything) is an object. Integers are objects, and so are Booleans. While Python isn't quite this object-oriented, objects are nonetheless abundant in the Python language. For example, the math.floor function is really just a class method for the math class. Class methods act on an entire class, rather than a particular instance of one. We often use them as utility functions.

Notice also that we've begun writing a class method for you. The `@classmethod` line that appears above it tells the Python interpreter that the method is a class method. You'll need to follow this pattern whenever you declare class methods.

In [4]:
import math

class Team():
    def __init__(self, team_name):
        self.team_name = team_name
        self.roster = []
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count
   
    def average_age(self):
        return math.fsum([player.age for player in self.roster]) / self.num_players()
    
    @classmethod
    def older_team(self, team1, team2):
        if (team1.average_age() > team2.average_age()):
            return team1
        else:
            return team2



In [5]:
old_team = Team.older_team(Team('New York Knicks'),Team('Miami Heat'))
print(old_team.team_name)

Miami Heat


## Inheritance 

In object-oriented programming, the concept of inheritance enables us to organize classes in a tree-like hierarchy, where the parent class has some traits that it passes on to its descendants. When we define a class, we specify a parent class from which it inherits. Inheriting from a class means that the behavior of the parent also exists in the child, but that the child can still define its own additional behavior.

Consider a Player class with generic information about NBA players. This would be very useful because players have a lot of things in common. However, we may also want to add specific behavior for different positions. We can define classes like Center, Forward, or Point Guard, each with behavior that's specific to that position. These classes would each specify Player as its parent class. They would all be siblings -- each would inherit the same behaviors from the Player class, while also having special behaviors of their own.

In Python 3, every class is a subclass of a generic object class. While this happens automatically when we don't specify an ancestor, it's sometimes good practice to be explicit. For the remainder of this mission, we'll specify when a class has object as its parent while we code. This is a good programming practice -- if we get into the habit of specifying a class's ancestry, we won't forget to specify a parent when it's something other than object. It's simply a way to form good habits.

### Overloading inherited behaviour

When a class inherits from a parent class, it acquires all of the behavior of that parent class. There are times when we don't want all of that behavior, though, or want to modify it slightly for our custom class. We use a technique called overloading to accomplish this.

Overloading inherited behavior involves assigning new behavior to our custom class. To accomplish this, we just redefine the method on our new class.

We'll be altering our Player class to support comparisons that use these operators:

These methods already exist in the object class by default, and we've used these operators to compare integers, floating point numbers (decimals), and strings. The operators work because classes like string have implemented them specifically. It's a bit difficult to understand why the object class would need to have these methods, however. The best way to wrap your head around this is through an example.

Let's consider the addition operator (+). The object class already defines a method for addition. The sum() function is defined using this addition method, but the object class doesn't really know how to add integers or floating points specifically.

However, the integer and float classes define their own addition method (thus overloading the object's addition method), and the sum() function will add the values together properly. This architecture is very powerful, because even though sum() only had to be defined once, we can call it on a multitude of classes and it will result in proper behavior. This is an example of the power of inheritance and overloading.

In [10]:
class Player(object):
    # The special __init__ function runs whenever a class is instantiated
    # The init function can take arguments, but self is always the first one
    # Self is just a reference to the instance of the class
    # It is automatically passed in when you instantiate an instance of the class
    def __init__(self, data_row):
        self.player_name = data_row[0]
        self.position = data_row[1]
        self.age = int(data_row[2])
        self.team = data_row[3]
    def __lt__(self, other):
        return self.age < other.age
    
    def __gt__(self,other):
        return self.age > other.age
    
    def __le__(self,other):
        return self.age <= other.age
    
    def __ge__(self,other):
        return self.age >= other.age
    
    def __eq__(self,other):
        return self.age == other.age
    
    def __ne__(self,other):
        return self.age != other.age
    # Implement the rest of the comparison operators here
    

carmelo = Player(nba[17])
kobe = Player(nba[68])

result = (carmelo != kobe)
print(result)

True


In [13]:
# Compare average ages of teams

import math

class Team(object):
    def __init__(self, team_name):
        self.team_name = team_name
        self.roster = []
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count
    def average_age(self):
        return math.fsum([player.age for player in self.roster]) / self.num_players()
    # Define operators here
    
    def __lt__(self,other):
        return self.average_age()<other.average_age()
    
    def __gt__(self,other):
        return self.average_age()<other.average_age()

    def __le__(self,other):
        return self.average_age() <= other.average_age()

    def __ge__(self,other):
        return self.average_age() >= other.average_age()

    def __eq__(self,other):
        return self.average_age() == other.average_age()

    def __ne__(self,other):
        return self.average_age() != other.average_age()

    
utah = Team('Utah Jazz')
detroit = Team('Detroit Pistons')
    
if utah < detroit:
    older_team = detroit
else: 
    older_team = utah
    
print(older_team.team_name)

Utah Jazz


### Min Max

A lot of interesting information is readily available to us now that we've implemented the comparison operations. That's because Python uses these comparisons to implement many utility functions. Now we're able to use those functions to analyze data in a new setting. By overloading methods, we've given ourselves access to powerful functions without having to implement tedious logic.

In [14]:
import math

class Team(object):
    def __init__(self, team_name):
        self.team_name = team_name
        self.roster = []
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count
    def average_age(self):
        return math.fsum([player.age for player in self.roster]) / self.num_players()
    def __lt__(self, other):
        return self.average_age() < other.average_age()
    def __gt__(self, other):
        return self.average_age() > other.average_age()
    def __le__(self, other):
        return self.average_age() <= other.average_age()
    def __ge__(self, other):
        return self.average_age() >= other.average_age()
    def __eq__(self, other):
        return self.average_age() == other.average_age()
    def __ne__(self, other):
        return self.average_age() != other.average_age()

team_names = ["Boston Celtics", "Brooklyn Nets", "New York Knicks", "Philadelphia 76ers", "Toronto Raptors", 
         "Chicago Bulls", "Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "Milwaukee Bucks",
         "Atlanta Hawks", "Charlotte Hornets", "Miami Heat", "Orlando Magic", "Washington Wizards",
         "Dallas Mavericks", "Houston Rockets", "Memphis Grizzlies", "New Orleans Pelicans", "San Antonio Spurs",
         "Denver Nuggets", "Minnesota Timberwolves", "Oklahoma City Thunder", "Portland Trail Blazers", "Utah Jazz",
         "Golden State Warriors", "Los Angeles Clippers", "Los Angeles Lakers", "Phoenix Suns", "Sacramento Kings"]

# Alter this list comprehension
teams = list([Team(name) for name in team_names])

oldest_team = max(teams)
youngest_team = min(teams)
sorted_teams = sorted(teams)

print(oldest_team.team_name)
print(youngest_team.team_name)

Miami Heat
Philadelphia 76ers
